<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/encoder_decoderV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pwd

'C:\\Users\\Hasibur1\\Desktop\\thisis\\final'

In [3]:
import pandas as pd
import numpy as np
import sys
import string
from string import digits
from pandas import Panel
from sklearn.datasets import make_circles
from matplotlib import pyplot
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import os
import re
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et 
import tensorflow as tf 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

C:\Users\Hasibur1\Anaconda3_\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  


In [4]:
# import pandas as pd 
import pandas as pd 
   
# Takes the file's folder 
#filepath = r"C:Documents\dataset.csv"

filepath = r"C:\Users\Hasibur1\Desktop\thisis\final\dataset.csv"
# read the CSV file 

#lines = pd.read_csv(filepath, encoding='ascii')
lines = pd.read_csv(filepath, sep='delimiter', header=None)

C:\Users\Hasibur1\Anaconda3_\lib\site-packages\ipykernel_launcher.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # This is added back by InteractiveShellApp.init_path()


In [5]:
print(len(lines)) 
lines.columns = ['qa']
list(lines.columns)

4040


['qa']

In [6]:
lines[['question','answer']] = lines.qa.apply(lambda x: pd.Series(str(x).split("?")))

In [7]:
list(lines.columns)

['qa', 'question', 'answer']

In [8]:
for col in lines.columns: 
    if 'qa' in col: 
        del lines[col] 
  

In [9]:
lines.iloc[:3]

,question,answer
0,What is (are) abdominal wall defect,",An opening in the abdomen through which vario..."
1,What are the treatments for abdominal wall defect,",Diagnostic Tests-Drug Therapy-Surgery and Reh..."
2,What is (are) Osteoporosis,",A Bone Disease that thins and weakens the bon..."


In [10]:
lines.columns = ['source', 'target']

In [11]:
list(lines.columns)

['source', 'target']

In [12]:
lines['source'].iloc[2000]

'What are the treatments for late-infantile neuronal ceroid lipofuscinosis '

In [13]:
lines['target'].iloc[2000]

',"An eye disorder that affects the clear, outer covering of the eye called the cornea. "'

In [14]:
lines['target'] = lines['target'].fillna('').apply(str)

In [15]:
# convert source and target text to Lowercase 
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())

# Remove quotes from source and target text
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.target=lines.target.apply(lambda x: re.sub("'", '', x))

#s = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", " ", x)

lines.source=lines.source.apply(lambda x: re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]',r' ',x))
lines.target=lines.target.apply(lambda x: re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]',r' ',x))

'''
# create a set of all special characters
special_characters= set(string.punctuation)

# Remove all the special characters
lines.source=lines.source.apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]',r'',x))
lines.target=lines.target.apply(lambda x: re.sub(r'[^a-zA-Z0-9 ]',r'',x))
'''
# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
lines.source=lines.source.apply(lambda x: x.translate(num_digits))
lines.target= lines.target.apply(lambda x: x.translate(num_digits))

# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))


In [16]:
# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')
lines.sample(6)

,source,target
435,what are the genetic changes related to multip...,START_ mutations in the tek gene cause vmcm _END
3488,how many people are affected by trichohepatoen...,START_ at least cases have been reported in th...
196,how many people are affected by mayer rokitans...,START_ mrkh syndrome affects approximately in ...
2267,how to diagnose polycythemia vera,START_ diagnosed by specific gene testing biop...
884,how many people are affected by ring chromosom...,START_ ring chromosome syndrome appears to be ...
3034,how many people are affected by sensorineural ...,START_ the prevalence of sensorineural deafnes...


In [17]:
lines['source'].iloc[2086]

'how to diagnose colon cancer'

In [18]:
lines['target'].iloc[2086]

'START_ digital rectal exam biopsy _END'

In [19]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for source in lines.source:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)
# Vocabulary of Target 
all_target_words=set()
for target in lines.target:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

len(source_words), len(target_words)

(1270, 3635)

In [20]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)
target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 19
 Max length of the target sentence 31


In [21]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [24]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
print(source_idx2word)
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

{1: 'a', 2: 'aarskog', 3: 'ab', 4: 'abdominal', 5: 'abetalipoproteinemia', 6: 'abnormal', 7: 'absent', 8: 'abuse', 9: 'acatalasemia', 10: 'accumulation', 11: 'acetylglutamate', 12: 'achondrogenesis', 13: 'achondroplasia', 14: 'achromatopsia', 15: 'acid', 16: 'acidemia', 17: 'acidosis', 18: 'aciduria', 19: 'acral', 20: 'acrofacial', 21: 'acromelic', 22: 'acromicric', 23: 'actin', 24: 'activated', 25: 'acute', 26: 'acyl', 27: 'adams', 28: 'adcy', 29: 'adenine', 30: 'adenosine', 31: 'adenylosuccinate', 32: 'adermatoglyphia', 33: 'adhesion', 34: 'adiposis', 35: 'adolescent', 36: 'adrenal', 37: 'adrenoleukodystrophy', 38: 'adult', 39: 'adults', 40: 'affected', 41: 'african', 42: 'agammaglobulinemia', 43: 'age', 44: 'aggravated', 45: 'aggregate', 46: 'aicardi', 47: 'aids', 48: 'alagille', 49: 'albinism', 50: 'albright', 51: 'aldrich', 52: 'alexander', 53: 'alg', 54: 'alkaptonuria', 55: 'allan', 56: 'allergic', 57: 'alopecia', 58: 'alpers', 59: 'alpha', 60: 'amelia', 61: 'american', 62: 'amyl

In [25]:
#Shuffle the data
lines = shuffle(lines)

In [26]:
# Train - Test Split
X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train.shape, X_test.shape

((2828,), (1212,))

In [27]:
# https://stackoverflow.com/questions/54880279/how-to-handle-invalid-argument-error-in-keras
# Input tokens for encoder
num_encoder_tokens=len(source_words) +1
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [28]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                        encoder_input_data[i, t] = source_word2idx[word] 
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [29]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 64
epochs = 100
latent_dim=256

In [30]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [31]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [34]:
# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [35]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    325376      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    930816      input_2[0][0]                    
____________________________________________________________________________________________

In [36]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size, epochs=epochs)

C:\Users\Hasibur1\Anaconda3_\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
44/44 [==============================] - 25s 575ms/step - loss: 2.1834 - accuracy: 0.1127 - val_loss: 2.0077 - val_accuracy: 0.1366
Epoch 2/100
44/44 [==============================] - 23s 515ms/step - loss: 1.8510 - accuracy: 0.1773 - val_loss: 1.6439 - val_accuracy: 0.2213
Epoch 3/100
44/44 [==============================] - 25s 557ms/step - loss: 1.6593 - accuracy: 0.2766 - val_loss: 1.7344 - val_accuracy: 0.3076
Epoch 4/100
44/44 [==============================] - 23s 522ms/step - loss: 1.4961 - accuracy: 0.3482 - val_loss: 1.5967 - val_accuracy: 0.3724
Epoch 5/100
44/44 [==============================] - 23s 520ms/step - loss: 1.3596 - accuracy: 0.4008 - val_loss: 1.5643 - val_accuracy: 0.4097
Epoch 6/100
44/44 [==============================] - 23s 514ms/step - loss: 1.2616 - accuracy: 0.4380 - val_loss: 1.2563 - val_accuracy: 0.4426
Epoch 7/100
44/44 [==============================] - 23s 523ms/step - loss: 1.1685 - accuracy: 0.4648 - val_loss: 1.2525 - val_accuracy:

44/44 [==============================] - 24s 556ms/step - loss: 0.0784 - accuracy: 0.9709 - val_loss: 1.4612 - val_accuracy: 0.5245
Epoch 58/100
44/44 [==============================] - 24s 545ms/step - loss: 0.0734 - accuracy: 0.9733 - val_loss: 1.5156 - val_accuracy: 0.5285
Epoch 59/100
44/44 [==============================] - 24s 556ms/step - loss: 0.0679 - accuracy: 0.9767 - val_loss: 1.1851 - val_accuracy: 0.5262
Epoch 60/100
44/44 [==============================] - 25s 567ms/step - loss: 0.0623 - accuracy: 0.9799 - val_loss: 1.4397 - val_accuracy: 0.5216
Epoch 61/100
44/44 [==============================] - 24s 551ms/step - loss: 0.0582 - accuracy: 0.9820 - val_loss: 1.3347 - val_accuracy: 0.5162
Epoch 62/100
44/44 [==============================] - 24s 555ms/step - loss: 0.0541 - accuracy: 0.9837 - val_loss: 1.4661 - val_accuracy: 0.5223
Epoch 63/100
44/44 [==============================] - 24s 548ms/step - loss: 0.0501 - accuracy: 0.9858 - val_loss: 1.3461 - val_accuracy: 0.522

In [33]:
model.save_weights('nmt_weights_0epochs.h5')

In [34]:
model.load_weights('nmt_weights_0epochs.h5')

In [35]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [36]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']
# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
# Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 110):
            stop_condition = True
# Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
# Update states
        states_value = [h, c]
    return decoded_sentence

In [44]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
i=-1

In [45]:
#Make predictions on the test dataset
i+= 1
for k in range(i,i+40):
    #print("\n",k)
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input Source Question:', X_test[k:k+1].values[0],'?')
    print('Actual Target Answer:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Target Answer:', decoded_sentence[:-4])
    print("\n")

Input Source Question: what are the treatments for todds paralysis ?
Actual Target Answer:  individuals must rest as comfortably as possible until the paralysis disappears 
Predicted Target Answer:  diagnostic tests drug therapy surgery and rehabilitation 


Input Source Question: what are the genetic changes related to glutamate formiminotransferase deficiency ?
Actual Target Answer:  mutations in the ftcd gene cause glutamate formiminotransferase deficiency 
Predicted Target Answer:  mutations in the abdomen gene cause that deficiency 


Input Source Question: what is are alpha reductase deficiency ?
Actual Target Answer:  a condition that affects male sexual development before birth and during puberty 
Predicted Target Answer:  a condition that prevents the body from converting certain fats to energy 


Input Source Question: what is the outlook for zellweger syndrome ?
Actual Target Answer:  loc syndrome is a disorder that leads to abnormalities of the voicebox finger and toenails 

In [39]:
#Make predictions on the train dataset

train_gen = generate_batch(X_train, y_train, batch_size = 1)
x=-1

In [40]:
x+=1
for j in range(x,x+40):
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input Source Question:', X_train[j:j+1].values[0],'?')
    print('Actual Target Answer:', y_train[j:j+1].values[0][6:-4])
    print('Predicted Target Answer:', decoded_sentence[:-4])
    print("\n")

Input Source Question: how many people are affected by intrauterine growth restriction metaphyseal dysplasia adrenal hypoplasia congenita and genital anomalies ?
Actual Target Answer:  image syndrome is very rare with only about cases reported in the medical literature 
Predicted Target Answer:  image syndrome is very rare with only about cases reported in the medical literature 


Input Source Question: how many people are affected by langer mesomelic dysplasia ?
Actual Target Answer:  the prevalence of langer mesomelic dysplasia is unknown 
Predicted Target Answer:  the prevalence of langer mesomelic dysplasia is unknown 


Input Source Question: what is are guanidinoacetate methyltransferase deficiency ?
Actual Target Answer:  an inherited disorder that primarily affects the brain and muscles 
Predicted Target Answer:  an inherited disorder that primarily affects the brain and muscles 


Input Source Question: how many people are affected by laing distal myopathy ?
Actual Target Ans